In [1]:
import requests
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from api_key_coingecko import API_KEY

In [2]:
url = f"https://api.coingecko.com/api/v3/coins/bitcoin/market_chart" #api end point 

days_to_analyse = 2
currency = 'usd'

headers = {
    "accept": "application/json",
    "x-cg-demo-api-key": API_KEY
}

params = {
    'vs_currency' : currency,
    'days' : str(days_to_analyse)
}

response = requests.get(url, params=params, headers=headers)
data = response.json()

weekday_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# ========== 1. Топ-5 криптовалют по капитализации (Pie Chart) ==========
def top_5_marketcap():
    url = f'{base_url}/coins/markets'
    params = {
        'vs_currency': currency,
        'order': 'market_cap_desc',
        'per_page': 5,
        'page': 1,
        'sparkline': 'false'
    }
    response = requests.get(url, headers=headers, params=params).json()

    labels = [coin['name'] for coin in response]
    values = [coin['market_cap'] for coin in response]

    plt.figure(figsize=(6, 6))
    plt.pie(values, labels=labels, autopct='%1.1f%%', startangle=140)
    plt.title('Top-5 Kryptowährungen nach Marktkapitalisierung (aktuell)')
    plt.tight_layout()
    plt.show()

In [3]:
def bitcoin_marketcap_history(days=365):
    url = f'{base_url}/coins/bitcoin/market_chart'
    params = {
        'vs_currency': currency,
        'days': days,
        'interval': 'daily'
    }
    response = requests.get(url, headers=headers, params=params).json()
    
    market_caps = response['market_caps']
    dates = [datetime.fromtimestamp(point[0] / 1000) for point in market_caps]
    values = [point[1] for point in market_caps]

    df = pd.DataFrame({'Date': dates, 'Market Cap': values})
    df.set_index('Date', inplace=True)

    plt.figure(figsize=(10, 5))
    plt.plot(df.index, df['Market Cap'] / 1e9, label='Bitcoin')
    plt.title('Капитализация Bitcoin за последний год')
    plt.ylabel('Капитализация (млрд $)')
    plt.xlabel('Дата')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
